### Imports 👽

In [ ]:
import pandas as pd
import sys
import os

sys.path.append(os.path.abspath(os.path.join('..', 'src')))
from data_classes.scenario import ScenarioSettings
from data_classes.enums import Scenario
import sampling

### Setup Scenario 🧪

In [14]:
SCENARIO = Scenario.SUM_SINES
settings = ScenarioSettings(SCENARIO)
input_dim = settings.INPUT_DIM
test_function = sampling.sum_sines_fun

### Generate Data 📊

In [15]:
qmc_train_x = sampling.generate_sobol_sequence(
    num_samples = 8192,
    dim = input_dim
)
qmc_train_y = test_function(qmc_train_x)

mc_train_x = sampling.generate_random_sequence(
    num_samples = 8192,
    dim = input_dim
)
mc_train_y = test_function(mc_train_x)

In [16]:
qmc_train_df = pd.DataFrame(
    data=qmc_train_x,
    columns=[f"input_{i+1}" for i in range(qmc_train_x.shape[1])]
)
qmc_train_df["solution"] = qmc_train_y

mc_train_df = pd.DataFrame(
    data=mc_train_x,
    columns=[f"input_{i+1}" for i in range(mc_train_x.shape[1])]
)
mc_train_df["solution"] = mc_train_y

### Store Data 💿

In [ ]:
output_dir = os.path.abspath(os.path.join('..', 'data', SCENARIO.value, 'input'))
os.makedirs(output_dir, exist_ok=True)

output_path = os.path.join(output_dir, 'qmc_sample.csv')
qmc_train_df.to_csv(output_path, index=False)

output_path = os.path.join(output_dir, 'mc_sample.csv')
mc_train_df.to_csv(output_path, index=False)